<a href="https://colab.research.google.com/github/yajuna/navier-stokes/blob/master/navier_stokes_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook implements the physics informed neural network methodology proposed by [Raissi et al](https://github.com/maziarraissi/PINNs), and is a Tensorflow 2 modification based on [Jan Blechschmidt and Oliver G. Ernst](https://github.com/janblechschmidt/PDEsByNNs/blob/main/PINN_Solver.ipynb).

In this notebook, we construct a physics informed neural network to solve the Navier-Stokes equation, and validate the methodology via
1. use the original data from Raissi's original work
2. use our own simulated data generated from OpenFoam

Finally, we apply the PINN methodology to measured data to compute the solution to Navier-Stokes equation.

### Import libraries. We mainly use Python's `Numpy` library, and the `Tensorflow` library. Visualization is performed by `Matplotlib` library.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas
import scipy.io

from time import time

tf.experimental.numpy.experimental_enable_numpy_behavior()

import sys
print("Python 3 version is", sys.version)
print("Numpy version is", np.__version__)
print("Tensorflow version is", tf.__version__)
import matplotlib
print("Matplotlib version is", matplotlib.__version__)

Python 3 version is 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Numpy version is 1.25.2
Tensorflow version is 2.15.0
Matplotlib version is 3.7.1


In [ ]:
A = np.array([[5,3],[3,2]])
b = np.array([[1],[1]])
sol = np.linalg.solve(A,b)
print(sol)

[[-1.]
 [ 2.]]


### Clone Raissi's repository and import data for training the neural network.

In [ ]:
!git clone https://github.com/maziarraissi/PINNs

fatal: destination path 'PINNs' already exists and is not an empty directory.


### Define path for Raissi's repository.

In [ ]:
import os
import sys

# "." for Colab/VSCode, and ".." for GitHub
repoPath = os.path.join(".", "PINNs")
# repoPath = os.path.join("..", "PINNs")
utilsPath = os.path.join(repoPath, "Utilities")
dataPath = os.path.join(repoPath, "main", "Data")
appDataPath = os.path.join(repoPath, "appendix", "Data")

### Define data type to be `float64`; set random seed for reproducibility.

In [ ]:
# Set data type
DTYPE='float64'
tf.keras.backend.set_floatx(DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

### Import data file `cylinder_nektar_wake.mat`. This set of high resolution data was generated by the spectral/hp-element solver NekTar. Implementation details is described [here](https://www.sciencedirect.com/science/article/abs/pii/S0021999118307125).

The data corresponds to the solution of the Navier-Stokes equation with the following boundary conditions of the [1, 8] by [-2,2] domain:

1. left boundary: uniform free stream velocity
2. right boundary: zero pressure outflow, located downstream of the cylinder
3. top and bottom: periodic condition

The notations for the dataset are explained below:

`N`: number of grid points in space
`T`: number of time steps

`U_star`: $x$- component of the velocity field $u$ and $y$- component of the velocity field $v$ values, for all U and T values. Read to `UU` and `VV`, and later reshaped to `u` and `v`. Each of size NT.

`P_star`: pressure $p$ for all N and T. Read to `PP`, then reshaped to `p`. Each of size NT.

`t`: time where data are recorded, of size T. Broadcasted to size NT.

`X_star`: spatial variables where data are recorded, including both $x$ and $y$. Read to `XX` and `YY`, then reshaped to `x` and `y`. Each broadcasted to size NT.

Training data was labeled `*_train`, with spatial, temporal, as well as velocity and pressure data. The training data was chosen randomly from the full dataset explained above, of size `N_train`.

Testing data was labeled `*_star`, chosen at a time step `snap`.

Input of the neural network is `X_train` and `data_train`, with $x$, $y$, and $t$ variables, as well as $u$, $v$, and $p$. Testing data is similar.

Training data consist of 5000 data points, chosen from NT points, denoted by `N_train`. Testing data was chosen at a random time snap, denoted by `snap`.

In [ ]:
N_train = 5000

path = os.path.join(dataPath, "cylinder_nektar_wake.mat")
data = scipy.io.loadmat(path)

# define training and testing data
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t.shape[0]

print('Size of x: N and size of t: T', N, T)

#tile independent variable Data (like meshgrid) to size N by T
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

######################################################################
######################## Noiseless Data ##############################
######################################################################
# Training Data
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]

u_train = u[idx,:]
v_train = v[idx,:]
p_train = p[idx,:]

# Test Data -- used in model.predict to test model
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]

# make training and testing data
X_train = tf.concat([x_train,y_train,t_train], axis = 1)
data_train = tf.concat([u_train,v_train,p_train], axis = 1) ###First occurrence of data_train

X_test = tf.concat([x_star,y_star,t_star], axis = 1)
data_test = tf.concat([u_star,v_star,p_star], axis = 1)

print(np.min(YY), np.max(YY), np.min(XX), np.max(XX))

Size of x: N and size of t: T 5000 200
-2.0 2.0 1.0 8.0


### We define the residual of the PDE. The physics information is given by the governing Navier-Stokes equations:

$u_t+(uu_x + vu_y)=-p_x+0.01(u_{xx}+u_{yy})$

$v_t+(uv_x + vv_y)=-p_y+0.01(v_{xx}+v_{yy})$,

here $(x,y)$ denotes the spatial variable, $t$ the temporal variable, $u(t,x,y)$ denotes the $x$-component of the velocity field, $v(t,x,y)$ the y-component, and $p(t,x,y)$ the pressure.

We include the following errors in the loss term of the neural network

$f = u_t+(uu_x + vu_y)+p_x-0.01(u_{xx}+u_{yy})$

$g = v_t+(uv_x + vv_y)+p_y-0.01(v_{xx}+v_{yy})$,

and we train the neural network by minimizing the loss including the above terms.



In [ ]:
# Define residual of the PDE
def fun_r(u, u_t, u_x, u_y, u_xx, u_yy, v, v_t, v_x, v_y, v_xx, v_yy, p_x, p_y):
  f = u_t + (u * u_x + v * u_y) + p_x - 0.01 * (u_xx + u_yy)
  g = v_t + (u * v_x + v * v_y) + p_y - 0.01 * (v_xx + v_yy)
  print('tf.concat([f,g], axis = 1).shape', tf.concat([f,g], axis = 1).shape)
  return tf.concat([f,g], axis = 1)

Initialize the neural network; by default, the neural network has 8 hidden layers, and 20 neurons on each layer.



In [ ]:
def init_model(num_hidden_layers=8, num_neurons_per_layer=20):
    # Initialize a feedforward neural network
    model = tf.keras.Sequential()

    # Input is three-dimensional (time + two spatial dimensions)
    model.add(tf.keras.Input(3))

    # # Define lower boundary and upper boundary
    # x = X_train[:,0:1]
    # y = X_train[:,1:2]
    # t = X_train[:,2:3]
    # lb = tf.constant([np.array(x.min()), np.array(y.min()), np.array(t.min())])
    # ub = tf.constant([np.array(x.max()), np.array(y.max()), np.array(t.max())])

    # Append hidden layers
    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,
            activation=tf.keras.activations.get('tanh'),
            kernel_initializer='glorot_normal'))

    # Output is two-dimensional: psi and p
    model.add(tf.keras.layers.Dense(2))

    return model

In [ ]:
def get_r(model, X_train):

    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split t and x to compute partial derivatives
        x, y, t = X_train[:, 0:1], X_train[:,1:2], X_train[:,2:3]

        # Variables x,y,t are watched during tape
        # to compute derivatives
        tape.watch(x)
        tape.watch(y)
        tape.watch(t)

        # Determine residual
        pp = model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))
        psi = pp[:,0:1]
        p = pp[:,1:2]
        # Compute gradients within the GradientTape
        # for all needed derivatives
        u = tape.gradient(psi, y)
        print("max and min of u", u.max, u.min)
        v = -1 * tape.gradient(psi, x)
        print("type of v", type(v))
        u_x = tape.gradient(u, x)
        print("max and min of u_x", u_x.max, u_x.min)
        u_y = tape.gradient(u, y)
        print("max and min of u_y", u_y.max, u_y.min)
        v_x = tape.gradient(v, x)
        print("max and min v_x", v_x.max, v_x.min)
        v_y = tape.gradient(v, y)



    p_x = tape.gradient(p, x)
    p_y = tape.gradient(p, y)

    u_t = tape.gradient(u, t)
    u_xx = tape.gradient(u_x, x)
    u_yy = tape.gradient(u_y, y)

    v_t = tape.gradient(v, t)
    v_xx = tape.gradient(v_x, x)
    v_yy = tape.gradient(v_y, y)

    del tape

    return fun_r(u, u_t, u_x, u_y, u_xx, u_yy, v, v_t, v_x, v_y, v_xx, v_yy, p_x, p_y)

In [ ]:
####### This is attempting to fix the shape error ##########

def compute_loss(model, X_train, data_train):

    # Compute phi^r
    r = get_r(model, X_train)
    phi_r = tf.reduce_mean(tf.square(r))

    # Initialize loss
    loss = phi_r
    # u_pred = model(X_train)
    with tf.GradientTape(persistent=True) as tape:
        # Split t and x to compute partial derivatives
        x, y, t = X_train[:, 0:1], X_train[:,1:2], X_train[:,2:3]

        # Variables x,y,t are watched during tape
        # to compute derivatives
        tape.watch(x)
        tape.watch(y)
        tape.watch(t)

        # Determine residual
        pp = model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))
        psi = pp[:,0:1]
        p = pp[:,1:2]

        u = tape.gradient(psi, y)
        v = -1 * tape.gradient(psi, x)
        del tape

        # Concatenate u, v, and p along with p to match the shape of data_train
        data_pred = tf.concat([u, v, p], axis=1)

    ################Added by Yoshi##################
    print("")
    print("################Added by Yoshi##################")
    print("This is data_pred.shape")
    print(data_pred.shape)
    print("")
    loss += tf.reduce_mean(tf.square(data_train - data_pred))

    return loss


In [ ]:
def get_grad(model, X_train, data_train):

    with tf.GradientTape(persistent=True) as tape:
        # This tape is for derivatives with
        # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model, X_train, data_train)

    g = tape.gradient(loss, model.trainable_variables)
    del tape

    return loss, g

In [ ]:
# Initialize model aka u_\theta
model = init_model()

# We choose a piecewise decay of the learning rate, i.e., the
# step size in the gradient descent type algorithm
# the first 1000 steps use a learning rate of 0.01
# from 1000 - 3000: learning rate = 0.001
# from 3000 onwards: learning rate = 0.0005

lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])

# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
from time import time

# Define one training step as a TensorFlow function to increase speed of training
@tf.function
def train_step():
    # Compute current loss and gradient w.r.t. parameters
    loss, grad_theta = get_grad(model, X_train, data_train)

    # Perform gradient descent step
    optim.apply_gradients(zip(grad_theta, model.trainable_variables))

    return loss

# Number of training epochs
N = 500
hist = []

# Start timer
t0 = time()

for i in range(N+1):

    loss = train_step()

    # Append current loss to hist
    hist.append(loss.numpy())

    # Output current loss after 50 iterates
    if i%50 == 0:
        print('It {:05d}: loss = {:10.8e}'.format(i,loss))

# Print computation time
print('\nComputation time: {} seconds'.format(time()-t0))

max and min of u <bound method amax of <tf.Tensor 'gradient_tape/strided_slice_4/StridedSliceGrad:0' shape=(5000, 1) dtype=float64>> <bound method amin of <tf.Tensor 'gradient_tape/strided_slice_4/StridedSliceGrad:0' shape=(5000, 1) dtype=float64>>
type of v <class 'tensorflow.python.framework.ops.SymbolicTensor'>
max and min of u_x <bound method amax of <tf.Tensor 'gradient_tape/strided_slice_3/StridedSliceGrad_2:0' shape=(5000, 1) dtype=float64>> <bound method amin of <tf.Tensor 'gradient_tape/strided_slice_3/StridedSliceGrad_2:0' shape=(5000, 1) dtype=float64>>
max and min of u_y <bound method amax of <tf.Tensor 'gradient_tape/strided_slice_4/StridedSliceGrad_3:0' shape=(5000, 1) dtype=float64>> <bound method amin of <tf.Tensor 'gradient_tape/strided_slice_4/StridedSliceGrad_3:0' shape=(5000, 1) dtype=float64>>
max and min v_x <bound method amax of <tf.Tensor 'gradient_tape/strided_slice_3/StridedSliceGrad_4:0' shape=(5000, 1) dtype=float64>> <bound method amin of <tf.Tensor 'gradie

In [ ]:
model.save("navier-stokes-pinn.keras")

In [ ]:
data_pred = model.predict(X_test)
# convert data_pred to tensor for gradientTape
data_tf = tf.convert_to_tensor(data_pred)

157/157 [==============================] - 0s 2ms/step


In [ ]:
with tf.GradientTape(persistent=True) as tape:

  x,y,t = X_test[:,0:1], X_test[:,1:2], X_test[:,2:3]
  tape.watch(x)
  tape.watch(y)
  data_pred = model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))
  psi_pred = data_pred[:,0:1]
  p_pred = data_pred[:,1:2]
u = tape.gradient(psi_pred, y)
print(type(u))
v = -1 * tape.gradient(psi_pred, x)
data_pred = tf.concat([u,v,p_pred], axis = 1)

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
# # understand how model and model.predict work
# p_model_predict = data_tf[:,1:2]
# error1 = tf.reduce_mean(tf.square(p_pred - p_model_predict))
# print(p_model_predict)
# print(p_pred)
# print(error1)

In [ ]:
error = tf.reduce_mean(tf.square(data_test - data_pred))
print(error)

tf.Tensor(0.01498067142718852, shape=(), dtype=float64)


### Visualization keeps crashing; should do this on HYAK

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import cm
# from matplotlib.ticker import LinearLocator, FormatStrFormatter
# import matplotlib.pyplot as plt
# # visualize predicted data and testing data at snap.
# x,y,t = X_test[:,0:1], X_test[:,1:2], X_test[:,2:3]
# X,Y = np.meshgrid(x, y) # grid of point
# Z = data_pred[:, 0:1] # evaluation of the function on the grid

# fig = plt.figure()
# ax = fig.add_subplot(projection = '3d')
# surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
#                       cmap=cm.RdBu,linewidth=0, antialiased=False)

# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# fig.colorbar(surf, shrink=0.5, aspect=5)

# plt.show()

In [ ]:
new_model = tf.keras.models.load_model('navier-stokes-pinn.keras')

# Show the model architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                80        
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 20)                420       
                                                                 
 dense_4 (Dense)             (None, 20)                420       
                                                                 
 dense_5 (Dense)             (None, 20)                420       
                                                                 
 dense_6 (Dense)             (None, 20)                4